In [2]:
from pyspark import SparkContext
from pyspark.sql import *

In [3]:
sc =SparkContext() 

In [6]:
sqlContext = SQLContext(sc)

In [14]:
data = sqlContext.read.format("csv").option("header","true").load("AppleStore.csv")

In [15]:
data.show()

+---+---------+--------------------+----------+--------+-----+----------------+----------------+-----------+---------------+-------+-----------+-----------------+---------------+---------------+--------+-------+
|_c0|       id|          track_name|size_bytes|currency|price|rating_count_tot|rating_count_ver|user_rating|user_rating_ver|    ver|cont_rating|      prime_genre|sup_devices.num|ipadSc_urls.num|lang.num|vpp_lic|
+---+---------+--------------------+----------+--------+-----+----------------+----------------+-----------+---------------+-------+-----------+-----------------+---------------+---------------+--------+-------+
|  1|281656475|     PAC-MAN Premium| 100788224|     USD| 3.99|           21292|              26|          4|            4.5|  6.3.5|         4+|            Games|             38|              5|      10|      1|
|  2|281796108|Evernote - stay o...| 158578688|     USD|    0|          161065|              26|          4|            3.5|  8.2.2|         4+|     Pro

In [23]:
data.select(data['size_bytes'].alias("GB")).show()

+---------+
|       GB|
+---------+
|100788224|
|158578688|
|100524032|
|128512000|
| 92774400|
| 10485713|
|227795968|
|130242560|
| 49250304|
| 70023168|
| 49618944|
|227547136|
|179979264|
|160925696|
| 55153664|
|207907840|
|389879808|
|167407616|
|147093504|
| 10735026|
+---------+
only showing top 20 rows



In [25]:
data.createOrReplaceTempView("applestore")

In [39]:
#top 10 apps based on rating count and user rating
sqlContext.sql('select track_name,rating_count_tot,user_rating from applestore order by user_rating desc,rating_count_tot desc').show(10)

+--------------------+----------------+-----------+
|          track_name|rating_count_tot|user_rating|
+--------------------+----------------+-----------+
|         Rogue Ninja|              99|          5|
|Word Swag - Cool ...|            9731|          5|
|                ASOS|            9725|          5|
|      Papers, Please|             970|          5|
|                Pitu|             968|          5|
|Butt Sworkit - Fr...|             960|          5|
|          """HOOK"""|             959|          5|
|Sniper Shooter Pr...|             945|          5|
|Evolution Calcula...|            9269|          5|
|Tank.IO War - Fre...|            9259|          5|
+--------------------+----------------+-----------+
only showing top 10 rows



In [79]:
#The difference in the average number of screenshots displayed of highest and lowest rating apps.
sqlContext.sql('with cte as( \
    select user_rating,avg(`ipadSc_urls.num`) as avg_sc\
                  from applestore \
                  group by user_rating \
                  order by user_rating)\
                  \
    select (b.sc2 -a.sc1) as avg_difference_screenshot from (\
               (select avg_sc as sc1 from cte where user_rating =(select min(user_rating) from cte)) a\
               full join   \
    (select avg_sc as sc2 from cte where user_rating =(select max(user_rating) from cte)) b on a.sc1!=b.sc2)').show()

+-------------------------+
|avg_difference_screenshot|
+-------------------------+
|       1.1310680248890757|
+-------------------------+



In [56]:
data.select(data['`ipadSc_urls.num`']).show()

+---------------+
|ipadSc_urls.num|
+---------------+
|              5|
|              5|
|              5|
|              5|
|              5|
|              5|
|              0|
|              4|
|              5|
|              0|
|              4|
|              0|
|              4|
|              0|
|              5|
|              1|
|              1|
|              5|
|              3|
|              0|
+---------------+
only showing top 20 rows



In [84]:
# apps with highest user rating that supports multiple languages 

sqlContext.sql('select track_name,\
               user_rating,\
               `lang.num`\
                     from applestore \
                     where user_rating in (select max(user_rating) from applestore)\
                        and `lang.num`>1').show()

+---------------------------------+-----------+--------+
|                       track_name|user_rating|lang.num|
+---------------------------------+-----------+--------+
|             TurboScan™ Pro - ...|          5|       9|
|               Plants vs. Zombies|          5|       5|
|             Learn to Speak Sp...|          5|       5|
|             Plants vs. Zombie...|          5|       5|
|                        ▻Sudoku +|          5|       7|
|                     Flashlight Ⓞ|          5|      22|
|                   Infinity Blade|          5|      13|
|             Learn English qui...|          5|       5|
|                            Meitu|          5|       5|
|             Bodyweight Traini...|          5|       5|
|                Bitauto Autoprice|          5|       3|
|             Chess Pro - with ...|          5|       3|
|             Kurumaki Calendar...|          5|      31|
|               Domino's Pizza USA|          5|       2|
|             5K Runner: 0 to 5

In [91]:
data.select(data['`sup_devices.num`'],data['`ipadSc_urls.num`'],data['`lang.num`'],data['`vpp_lic`']).summary().show()

+-------+------------------+------------------+-----------------+-------------------+
|summary|   sup_devices.num|   ipadSc_urls.num|         lang.num|            vpp_lic|
+-------+------------------+------------------+-----------------+-------------------+
|  count|              7197|              7197|             7197|               7197|
|   mean| 37.36181742392664|3.7071001806308184| 5.43490343198555| 0.9930526608309017|
| stddev|3.7377152388584527|1.9860046449596336|7.919592722881359|0.08306643356297923|
|    min|                11|                 0|                0|                  0|
|    25%|              37.0|               3.0|              1.0|                1.0|
|    50%|              37.0|               5.0|              1.0|                1.0|
|    75%|              38.0|               5.0|              8.0|                1.0|
|    max|                 9|                 5|                9|                  1|
+-------+------------------+------------------+-------